So far, there are two objects:
- Sym_UTB_Matrix, which represents a direct sum of symbolic matrices (where each matrix in the direct sum is assumed to be an upper triangular block matrices)
- UTB_Subspace, which represents a particular kind of subspace of a direct sum of upper triangular block matrices. In particular, the subspace is specified as: every block is either 0, can take any scalar value (i.e. a scalar times the identity), or can take the value of any matrix (of the appropariate dimension)

In [1]:
from Sym_UTB_Matrix import Sym_UTB_Matrix
from UTB_Subspace import UTB_Subspace

### Sym_UTB_Matrix - Simple example

Consider a simple example of these symbolic matrices, e.g. the generators for computing two first order Dyson terms:

In [5]:
first_order = Sym_UTB_Matrix([[['G','A'],[0,'G']], [['G','B'],[0,'G']]])
first_order.mat_list

[Matrix([
 [G, A],
 [0, G]]), Matrix([
 [G, B],
 [0, G]])]

We can get a concise description of this matrix, i.e. a list of the unique symbols, along with their locations:

In [6]:
first_order.concise_display()

Shape:
------
[2, 2]
Non-commutative symbols:
------------------------
G {(1, 0, 0), (0, 1, 1), (1, 1, 1), (0, 0, 0)}
A {(0, 0, 1)}
B {(1, 0, 1)}
Commutative symbols:
--------------------


The above says: 
- the shape is [2,2], i.e. each matrix in the direct sum has block dimension 2
- the unique non-commutative symbols are 'G', 'A', and 'B', and their locations are listed, where for the index (i, j, k), i represents which part of the direct sum it lives in, and (j,k) is the location of the block in that direct sum
- Here, there are no commutative symbols (scalars)

We can define another matrix of this form, and multiply them

In [7]:
first_order2 = Sym_UTB_Matrix([[['H','C'],[0,'H']], [['H','D'],[0,'H']]])
first_product = first_order.mult(first_order2)
first_product.concise_display()

Shape:
------
[2, 2]
Non-commutative symbols:
------------------------
G*H {(1, 0, 0), (0, 1, 1), (1, 1, 1), (0, 0, 0)}
A*H + G*C {(0, 0, 1)}
B*H + G*D {(1, 0, 1)}
Commutative symbols:
--------------------


The above shows the unique symbols in the product of the two, which in this case gives exactly a symbolic description of the reduced multiplication rules we are ultimately after.

We can also add, or take powers of them:

In [9]:
first_order.add(first_order2).concise_display()
first_order.power(2).concise_display()

Shape:
------
[2, 2]
Non-commutative symbols:
------------------------
G + H {(1, 0, 0), (0, 1, 1), (1, 1, 1), (0, 0, 0)}
A + C {(0, 0, 1)}
B + D {(1, 0, 1)}
Commutative symbols:
--------------------
Shape:
------
[2, 2]
Non-commutative symbols:
------------------------
G**2 {(1, 0, 0), (0, 1, 1), (1, 1, 1), (0, 0, 0)}
A*G + G*A {(0, 0, 1)}
B*G + G*B {(1, 0, 1)}
Commutative symbols:
--------------------


### UTB_Subspace - simple example

Next, consider the UTB_Subspace object. Here, we consider the form of a second order generator, and display it.

In [11]:
second_subspace = UTB_Subspace.from_matrix([['G', 'A', 0], [0,'G','B'], [0,0,'G']])
second_subspace.display()

Unique non-commutative blocks:
------------------------------
1. frozenset({(0, 1, 1), (0, 0, 0), (0, 2, 2)})
2. frozenset({(0, 0, 1)})
3. frozenset({(0, 1, 2)})
Unique commutative blocks:
--------------------------
Zero blocks:
------------
frozenset({(0, 0, 2)})


Note: what the matrix [['G', 'A', 0], [0,'G','B'], [0,0,'G']] represents when used to construct this object, is the subspace of 3x3 UTB matrices where the diagonal blocks are the same (but could be any matrix), the first off diagonal blocks can be any matrix, and the top rigth block is 0.

As this object is supposed to represent a subspace, it "forgets" the symbols used, all it does is keep track of sets of indices that represent the same object (either a matrix, a scalar, or a zero block). (Note that we are storing them as frozensets due to implementation details.) The above description says that the diagonals are all the same matrix, and the first off diagonals are free to vary as any matrix, the top rigth block is 0, and the there are no commutative blocks (i.e. no blocks represented by scalars).

The methods of this object are meant to represent manipulations one would normally want to consider for subspace. E.g. we can add subspaces to get a new subspace, multiply subspaces to get a new subspace, check if one subspace is a subset of another, or if they are equal.

Importantly for us, we can check if the subspace also happens to be an algebra:

In [12]:
second_subspace.is_algebra()

False

This subspace is not an algebra, i.e. if you multiply matrices in this subspace together, you get matrices outside of the subspace. Specifically, the top right block will become non-zero when these are multiplied together. We can see this by generating symbolic elements of the subspace, and multiplying them together.

In [14]:
mat1 = second_subspace.get_subspace_member(['A0','A1','A2'])
mat2 = second_subspace.get_subspace_member(['B0','B1','B2'])

print('mat1')
mat1.concise_display()
print('mat2')
mat2.concise_display()

print('mat1 mult mat2')
mat1.mult(mat2).concise_display()

mat1
Shape:
------
[3]
Non-commutative symbols:
------------------------
A0 {(0, 1, 1), (0, 0, 0), (0, 2, 2)}
A1 {(0, 0, 1)}
A2 {(0, 1, 2)}
Commutative symbols:
--------------------
0 {(0, 0, 2)}
mat2
Shape:
------
[3]
Non-commutative symbols:
------------------------
B0 {(0, 1, 1), (0, 0, 0), (0, 2, 2)}
B1 {(0, 0, 1)}
B2 {(0, 1, 2)}
Commutative symbols:
--------------------
0 {(0, 0, 2)}
mat1 mult mat2
Shape:
------
[3]
Non-commutative symbols:
------------------------
A0*B0 {(0, 1, 1), (0, 0, 0), (0, 2, 2)}
A0*B1 + A1*B0 {(0, 0, 1)}
A0*B2 + A2*B0 {(0, 1, 2)}
A1*B2 {(0, 0, 2)}
Commutative symbols:
--------------------


Since the subspace is not an algebra, we may want to retrieve the algebra it lives in:

In [17]:
second_alg = second_subspace.containing_algebra()
second_alg.display()

Unique non-commutative blocks:
------------------------------
1. frozenset({(0, 1, 1), (0, 0, 0), (0, 2, 2)})
2. frozenset({(0, 0, 1)})
3. frozenset({(0, 1, 2)})
4. frozenset({(0, 0, 2)})
Unique commutative blocks:
--------------------------
Zero blocks:
------------
frozenset()


We can see that the description of second_alg is similar to second_subspace, except the top rigth block has been moved from a zero block, to a block that can take the value of any matrix.

Lastly, we want to see how to more efficiently multiply matrices in this algebra. In particular, this algebra contains elements with 4 unique blocks, so we need to extract the rules for multiplying these matrices. (The interface for this is something I need to work on, but doing it with the existing code only requires a few lines.) To do this, we get two arbitrary symbolic representatives from the algebra, and multiply them together:

In [20]:
mat1 = second_alg.get_subspace_member('A')
mat2 = second_alg.get_subspace_member('B')

print('first member:')
print(mat1.mat_list)
print('second member:')
print(mat2.mat_list)

print('multiplication rules:')
mat3 = mat1.mult(mat2)
mat3.concise_display()

first member:
[Matrix([
[A0, A1, A3],
[ 0, A0, A2],
[ 0,  0, A0]])]
second member:
[Matrix([
[B0, B1, B3],
[ 0, B0, B2],
[ 0,  0, B0]])]
multiplication rules:
Shape:
------
[3]
Non-commutative symbols:
------------------------
A0*B0 {(0, 1, 1), (0, 0, 0), (0, 2, 2)}
A0*B1 + A1*B0 {(0, 0, 1)}
A0*B2 + A2*B0 {(0, 1, 2)}
A0*B3 + A1*B2 + A3*B0 {(0, 0, 2)}
Commutative symbols:
--------------------


In the above, the reduced multiplication rules are given by the non-commutative symbols

### More complicated example
Consider the case where we have X and Y control of a qubit, and want to compute the first order dyson terms for all Paulis: X, Y, and Z. Furthermore, consider the generators for computing these terms, along with the derivatives with of these terms with respect to X and Y.

In [25]:
deriv_system = Sym_UTB_Matrix([[['G','X','X',0], [0, 'G',0,'X'],[0,0,'G','X'],[0,0,0,'G']],[['G','X','Y',0], [0, 'G',0,'Y'],[0,0,'G','X'],[0,0,0,'G']], [['G','Y','X',0], [0, 'G',0,'X'],[0,0,'G','Y'],[0,0,0,'G']], [['G','Y','Y',0], [0, 'G',0,'Y'],[0,0,'G','Y'],[0,0,0,'G']], [['G','Z','X',0], [0, 'G',0,'X'],[0,0,'G','Z'],[0,0,0,'G']], [['G','Z','Y',0], [0, 'G',0,'Y'],[0,0,'G','Z'],[0,0,0,'G']]   ])
print(deriv_system.mat_list)

[Matrix([
[G, X, X, 0],
[0, G, 0, X],
[0, 0, G, X],
[0, 0, 0, G]]), Matrix([
[G, X, Y, 0],
[0, G, 0, Y],
[0, 0, G, X],
[0, 0, 0, G]]), Matrix([
[G, Y, X, 0],
[0, G, 0, X],
[0, 0, G, Y],
[0, 0, 0, G]]), Matrix([
[G, Y, Y, 0],
[0, G, 0, Y],
[0, 0, G, Y],
[0, 0, 0, G]]), Matrix([
[G, Z, X, 0],
[0, G, 0, X],
[0, 0, G, Z],
[0, 0, 0, G]]), Matrix([
[G, Z, Y, 0],
[0, G, 0, Y],
[0, 0, G, Z],
[0, 0, 0, G]])]


Above are the generators for doing this. These obviously have a lot of redundancies. Furthermore, you can possibly convince yourself that the 0 block in the (1, 2) position (using indexing starting at 0) will always be zero regardless of how many powers or linear combinations of matrices you take of this form. The concise description is:

In [26]:
deriv_system.concise_display()

Shape:
------
[4, 4, 4, 4, 4, 4]
Non-commutative symbols:
------------------------
G {(0, 1, 1), (4, 2, 2), (1, 0, 0), (4, 3, 3), (5, 2, 2), (3, 3, 3), (4, 0, 0), (2, 3, 3), (5, 1, 1), (0, 2, 2), (3, 2, 2), (2, 0, 0), (0, 3, 3), (3, 1, 1), (1, 2, 2), (5, 3, 3), (0, 0, 0), (4, 1, 1), (1, 1, 1), (5, 0, 0), (2, 2, 2), (2, 1, 1), (3, 0, 0), (1, 3, 3)}
X {(4, 1, 3), (0, 0, 2), (0, 2, 3), (2, 0, 2), (0, 0, 1), (1, 2, 3), (1, 0, 1), (0, 1, 3), (4, 0, 2), (2, 1, 3)}
Y {(3, 0, 2), (3, 2, 3), (1, 1, 3), (2, 2, 3), (5, 0, 2), (2, 0, 1), (3, 1, 3), (3, 0, 1), (1, 0, 2), (5, 1, 3)}
Z {(4, 0, 1), (5, 2, 3), (5, 0, 1), (4, 2, 3)}
Commutative symbols:
--------------------
0 {(1, 0, 3), (5, 1, 2), (2, 0, 3), (3, 0, 3), (0, 1, 2), (1, 1, 2), (4, 0, 3), (5, 0, 3), (2, 1, 2), (4, 1, 2), (3, 1, 2), (0, 0, 3)}


I.e. there are 4 unique symbols representing these matrices: {G, X, Y, Z}, and their positions are given above.

Next, we want the algebra that matrices of the above form are living in. To do this, we construct a subspace from the above description, and then retrieve the algebra containing it.

In [27]:
deriv_subspace = UTB_Subspace.from_Sym_UTB_Matrix(deriv_system)
deriv_alg = deriv_subspace.containing_algebra()

We can see the structure of the algebra by looking at a representative:

In [28]:
alg_member = deriv_alg.get_subspace_member()
alg_member.mat_list

[Matrix([
 [A0, A1, A1, A4],
 [ 0, A0,  0, A1],
 [ 0,  0, A0, A1],
 [ 0,  0,  0, A0]]), Matrix([
 [A0, A1, A2, A5],
 [ 0, A0,  0, A2],
 [ 0,  0, A0, A1],
 [ 0,  0,  0, A0]]), Matrix([
 [A0, A2, A1, A5],
 [ 0, A0,  0, A1],
 [ 0,  0, A0, A2],
 [ 0,  0,  0, A0]]), Matrix([
 [A0, A2, A2, A6],
 [ 0, A0,  0, A2],
 [ 0,  0, A0, A2],
 [ 0,  0,  0, A0]]), Matrix([
 [A0, A3, A1, A7],
 [ 0, A0,  0, A1],
 [ 0,  0, A0, A3],
 [ 0,  0,  0, A0]]), Matrix([
 [A0, A3, A2, A8],
 [ 0, A0,  0, A2],
 [ 0,  0, A0, A3],
 [ 0,  0,  0, A0]])]

Observe: the generated algebra now has non-zero blocks in the top right, which are themselves unique for each of the matrices. Furthermore, the (1,2) spot remains a 0. The description of the algebra:

In [29]:
deriv_alg.display()

Unique non-commutative blocks:
------------------------------
1. frozenset({(0, 1, 1), (0, 3, 3), (4, 2, 2), (3, 1, 1), (1, 0, 0), (1, 2, 2), (5, 3, 3), (0, 0, 0), (4, 3, 3), (4, 1, 1), (1, 1, 1), (5, 2, 2), (5, 0, 0), (3, 3, 3), (2, 2, 2), (4, 0, 0), (2, 3, 3), (2, 1, 1), (5, 1, 1), (0, 2, 2), (3, 2, 2), (3, 0, 0), (2, 0, 0), (1, 3, 3)})
2. frozenset({(4, 1, 3), (0, 0, 2), (0, 2, 3), (2, 0, 2), (0, 0, 1), (1, 2, 3), (1, 0, 1), (0, 1, 3), (4, 0, 2), (2, 1, 3)})
3. frozenset({(3, 0, 2), (3, 2, 3), (1, 1, 3), (2, 2, 3), (5, 0, 2), (2, 0, 1), (3, 1, 3), (3, 0, 1), (1, 0, 2), (5, 1, 3)})
4. frozenset({(4, 0, 1), (5, 2, 3), (5, 0, 1), (4, 2, 3)})
5. frozenset({(0, 0, 3)})
6. frozenset({(1, 0, 3), (2, 0, 3)})
7. frozenset({(3, 0, 3)})
8. frozenset({(4, 0, 3)})
9. frozenset({(5, 0, 3)})
Unique commutative blocks:
--------------------------
Zero blocks:
------------
frozenset({(5, 1, 2), (2, 1, 2), (0, 1, 2), (4, 1, 2), (3, 1, 2), (1, 1, 2)})


I.e. there are 9 unique blocks with location given above, and the locations of the zero blocks are also listed. 

Finally, we can get the multiplication rules as before; get two symbolic representatives from this algebra, multiply them together, and the custom multiplication rules are encoded in the symbolic expression:

In [30]:
member1 = deriv_alg.get_subspace_member('A')
member2 = deriv_alg.get_subspace_member('B')

member1.mult(member2).concise_display()

Shape:
------
[4, 4, 4, 4, 4, 4]
Non-commutative symbols:
------------------------
A0*B0 {(0, 1, 1), (4, 2, 2), (1, 0, 0), (4, 3, 3), (5, 2, 2), (3, 3, 3), (4, 0, 0), (2, 3, 3), (5, 1, 1), (0, 2, 2), (3, 2, 2), (2, 0, 0), (0, 3, 3), (3, 1, 1), (1, 2, 2), (5, 3, 3), (0, 0, 0), (4, 1, 1), (1, 1, 1), (5, 0, 0), (2, 2, 2), (2, 1, 1), (3, 0, 0), (1, 3, 3)}
A0*B1 + A1*B0 {(4, 1, 3), (0, 0, 2), (0, 2, 3), (2, 0, 2), (0, 0, 1), (1, 2, 3), (1, 0, 1), (0, 1, 3), (4, 0, 2), (2, 1, 3)}
A0*B2 + A2*B0 {(3, 0, 2), (3, 2, 3), (1, 1, 3), (2, 2, 3), (5, 0, 2), (2, 0, 1), (3, 1, 3), (3, 0, 1), (1, 0, 2), (5, 1, 3)}
A0*B3 + A3*B0 {(4, 0, 1), (5, 2, 3), (5, 0, 1), (4, 2, 3)}
A0*B4 + 2*A1*B1 + A4*B0 {(0, 0, 3)}
A0*B5 + A1*B2 + A2*B1 + A5*B0 {(1, 0, 3), (2, 0, 3)}
A0*B6 + 2*A2*B2 + A6*B0 {(3, 0, 3)}
A0*B7 + A1*B3 + A3*B1 + A7*B0 {(4, 0, 3)}
A0*B8 + A2*B3 + A3*B2 + A8*B0 {(5, 0, 3)}
Commutative symbols:
--------------------
0 {(5, 1, 2), (0, 1, 2), (1, 1, 2), (2, 1, 2), (4, 1, 2), (3, 1, 2)}


So, the above shows that to work with these huge block matrices, we only need to actually keep track of and manipulate 9 unique blocks, with the multiplication rules given above.

Note: there are some things I need to be more careful with in terms of preserving symbol ordering and what not, but it all seems to be working.

#### Polynomial test

In [5]:
poly11_gen = Sym_UTB_Matrix([
                        ['G', 0, 'A', 0, 0, 0, 0],
                        [0, 'G', 0, 'A', 0, 0, 0],
                        [0, 0, 'G', 1, 'B', 0, 0],
                        [0, 0, 0, 'G', 0, 'B', 0],
                        [0, 0, 0, 0, 'G', 2, 0],
                        [0, 0, 0, 0, 0, 'G', 1],
                        [0, 0, 0, 0, 0, 0, 'G']])
poly11_sub = UTB_Subspace.from_Sym_UTB_Matrix(poly11_gen)
poly11_alg = poly11_sub.containing_algebra()
poly11_alg.display()

Unique non-commutative blocks:
------------------------------
1. frozenset({(0, 1, 1), (0, 3, 3), (0, 5, 5), (0, 0, 0), (0, 6, 6), (0, 4, 4), (0, 2, 2)})
2. frozenset({(0, 2, 3), (0, 5, 6)})
3. frozenset({(0, 4, 5)})
4. frozenset({(0, 0, 2), (0, 1, 3)})
5. frozenset({(0, 3, 5), (0, 2, 4)})
6. frozenset({(0, 4, 6)})
7. frozenset({(0, 0, 3)})
8. frozenset({(0, 2, 5)})
9. frozenset({(0, 3, 6)})
10. frozenset({(0, 1, 5), (0, 0, 4)})
11. frozenset({(0, 2, 6)})
12. frozenset({(0, 0, 5)})
13. frozenset({(0, 1, 6)})
14. frozenset({(0, 0, 6)})
Unique commutative blocks:
--------------------------
Zero blocks:
------------
frozenset({(0, 1, 2), (0, 1, 4), (0, 0, 1), (0, 3, 4)})


In [8]:
poly11_alg.get_subspace_member('A','a').mult(poly11_alg.get_subspace_member('B','b')).concise_display()

Shape:
------
[7]
Non-commutative symbols:
------------------------
A0*B0 {(0, 1, 1), (0, 3, 3), (0, 5, 5), (0, 4, 4), (0, 2, 2), (0, 6, 6), (0, 0, 0)}
A0*B1 + A1*B0 {(0, 2, 3), (0, 5, 6)}
A0*B2 + A2*B0 {(0, 4, 5)}
A0*B3 + A3*B0 {(0, 0, 2), (0, 1, 3)}
A0*B4 + A4*B0 {(0, 3, 5), (0, 2, 4)}
A0*B5 + A2*B1 + A5*B0 {(0, 4, 6)}
A0*B6 + A3*B1 + A6*B0 {(0, 0, 3)}
A0*B7 + A1*B4 + A4*B2 + A7*B0 {(0, 2, 5)}
A0*B8 + A4*B1 + A8*B0 {(0, 3, 6)}
A0*B9 + A3*B4 + A9*B0 {(0, 1, 5), (0, 0, 4)}
A0*B10 + A1*B8 + A10*B0 + A4*B5 + A7*B1 {(0, 2, 6)}
A0*B11 + A11*B0 + A3*B7 + A6*B4 + A9*B2 {(0, 0, 5)}
A0*B12 + A12*B0 + A3*B8 + A9*B1 {(0, 1, 6)}
A0*B13 + A11*B1 + A13*B0 + A3*B10 + A6*B8 + A9*B5 {(0, 0, 6)}
Commutative symbols:
--------------------
0 {(0, 1, 2), (0, 1, 4), (0, 0, 1), (0, 3, 4)}


### cross talk type computations

In [14]:
xtalk_sub = UTB_Subspace.from_matrix([[['X','Y'],[0,'X']], [['Y','X'],[0,'Y']]])
test_alg = test.containing_algebra()
test2 = Sym_UTB_Matrix([[['X','Y'],[0,'X']], [['Y','X'],[0,'Y']]])
test_alg.display()

Unique non-commutative blocks:
------------------------------
1. frozenset({(0, 1, 1), (0, 0, 0)})
2. frozenset({(1, 0, 0), (1, 1, 1)})
3. frozenset({(0, 0, 1)})
4. frozenset({(1, 0, 1)})
Unique commutative blocks:
--------------------------
Zero blocks:
------------
frozenset()


In [15]:
test.is_algebra()

False